Thay thế CNN thuần bằng ResNet-18 để tăng hiệu suất
Lý do chọn ResNet:

    ResNet sử dụng Residual Connections (Skip Connections) giúp tránh hiện tượng vanishing gradient.
    
    Có khả năng học được đặc trưng sâu hơn mà không bị mất thông tin.
    
    Kiến trúc đã được chứng minh là mạnh hơn CNN thuần trong nhiều bài toán hình ảnh.

In [ ]:
#Sử dụng ResNet-18 từ Keras với trọng số pre-trained trên ImageNet.

#Sử dụng ResNet-18 từ Keras với trọng số pre-trained trên ImageNet.

#Thay thế lớp Fully Connected cuối cùng để phù hợp với bài toán binary classification (Tumor/Normal).

import os
import socket
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import cv2

IMG_SIZE = 224  # ResNet yêu cầu input (224x224)

# Hàm kiểm tra internet
def check_internet():
    try:
        socket.create_connection(("www.google.com", 80))
        return True
    except OSError:
        return False

# Load dữ liệu
def load_data(json_folder, image_folder):
    images, labels = [], []
    json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]
    
    for json_file in json_files:
        json_path = os.path.join(json_folder, json_file)
        with open(json_path, "r") as file:
            data = json.load(file)
        
        image_file = data.get("imagePath")
        if not image_file:
            continue
        
        image_path = os.path.join(image_folder, image_file)
        if not os.path.exists(image_path):
            continue
        
        image = cv2.imread(image_path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = image / 255.0  # Chuẩn hóa
        images.append(image)
        
        label = 1 if "tumor" in data.get("shapes", [{}])[0].get("label", "").lower() else 0
        labels.append(label)
    
    return np.array(images), np.array(labels)

# Load dữ liệu
json_folder = "/kaggle/input/btxrd-data/BTXRD/BTXRD/Annotations"
image_folder = "/kaggle/input/btxrd-data/BTXRD/BTXRD/images"
X, y = load_data(json_folder, image_folder)

# Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Đường dẫn local chứa trọng số ResNet-50
local_weights_path = "/kaggle/input/resnet50-weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Kiểm tra internet
if check_internet():
    print("Có kết nối mạng, tải trọng số từ TensorFlow...")
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
elif os.path.exists(local_weights_path):
    print("Không có mạng, nhưng tìm thấy trọng số local. Đang tải...")
    base_model = ResNet50(weights=local_weights_path, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
else:
    print("Không có mạng và không tìm thấy trọng số local. Sử dụng weights=None...")
    base_model = ResNet50(weights=None, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Đóng băng trọng số ResNet-50
for layer in base_model.layers:
    layer.trainable = False

# Xây dựng mô hình
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile mô hình
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Đánh giá mô hình
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")


Loss function

In [ ]:
# Mô hình Focal loss
import os
import json
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K

IMG_SIZE = 48  # Giống với mô hình nhận diện cảm xúc

def load_data(json_folder, image_folder):
    images, labels = [], []
    json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]
    
    for json_file in json_files:
        json_path = os.path.join(json_folder, json_file)
        with open(json_path, "r") as file:
            data = json.load(file)
        
        image_file = data.get("imagePath")
        if not image_file:
            continue
        
        image_path = os.path.join(image_folder, image_file)
        if not os.path.exists(image_path):
            continue
        
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = image / 255.0  # Chuẩn hóa
        images.append(image)
        
        # Gán nhãn: 1 nếu label chứa từ "tumor", ngược lại 0
        label = 1 if "tumor" in data.get("shapes", [{}])[0].get("label", "").lower() else 0
        labels.append(label)
    
    return np.array(images).reshape(-1, IMG_SIZE, IMG_SIZE, 1), np.array(labels)
n)
json_folder = "/kaggle/input/btxrd-data/BTXRD/BTXRD/Annotations"
image_folder = "/kaggle/input/btxrd-data/BTXRD/BTXRD/images"
X, y = load_data(json_folder, image_folder)

# Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình CNN (dựa trên 3_Emotion_Reg_CNN.ipynb) với lớp Input đầu tiên
model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 1)),
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(128, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(256, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Phân loại nhị phân
])

# Định nghĩa Focal Loss
def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        return K.mean(alpha * K.pow((1 - p_t), gamma) * bce)
    return loss

# Compile mô hình với Focal Loss
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss=focal_loss(alpha=0.25, gamma=2.0),
              metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Đánh giá mô hình
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

# Hiển thị một số kết quả 
preds = model.predict(X_test)
preds_labels = (preds > 0.5).astype("int32")
fig, axes = plt.subplots(3, 3, figsize=(8, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_test[i].reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
    ax.set_title(f"Pred: {'Tumor' if preds_labels[i] == 1 else 'Normal'}\nConfidence: {preds[i][0]:.2f}")
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
#Test
import os
import json
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K

IMG_SIZE = 48  # Giống với mô hình nhận diện cảm xúc

# Hàm tải dữ liệu từ JSON
def load_data(json_folder, image_folder):
    images, labels = [], []
    json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]
    
    for json_file in json_files:
        json_path = os.path.join(json_folder, json_file)
        with open(json_path, "r") as file:
            data = json.load(file)
        
        image_file = data.get("imagePath")
        if not image_file:
            continue
        
        image_path = os.path.join(image_folder, image_file)
        if not os.path.exists(image_path):
            continue
        
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = image / 255.0  # Chuẩn hóa
        images.append(image)
        
        # Gán nhãn: 1 nếu label chứa từ "tumor", ngược lại 0
        label = 1 if "tumor" in data.get("shapes", [{}])[0].get("label", "").lower() else 0
        labels.append(label)
    
    return np.array(images).reshape(-1, IMG_SIZE, IMG_SIZE, 1), np.array(labels)

json_folder = "/kaggle/input/btxrd-data/BTXRD/BTXRD/Annotations"
image_folder = "/kaggle/input/btxrd-data/BTXRD/BTXRD/images"
X, y = load_data(json_folder, image_folder)

# Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình CNN với lớp Input đầu tiên
model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 1)),
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(128, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(256, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Phân loại nhị phân
])

# Định nghĩa Dice Loss
def dice_loss(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.cast(K.flatten(y_true), 'float32')
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Định nghĩa Dice Coefficient
def dice_coefficient(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.cast(K.flatten(y_true), 'float32')
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Compile mô hình với Dice Loss và thêm Dice Coefficient vào metrics
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss=dice_loss,
              metrics=['accuracy', dice_coefficient])

# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Đánh giá mô hình
loss, acc, dice_coef = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")
print(f"Dice Coefficient: {dice_coef:.2f}")

# Hiển thị một số kết quả 
preds = model.predict(X_test)
preds_labels = (preds > 0.5).astype("int32")
fig, axes = plt.subplots(3, 3, figsize=(8, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_test[i].reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
    ax.set_title(f"Pred: {'Tumor' if preds_labels[i] == 1 else 'Normal'}\nConf: {preds[i][0]:.2f}")
    ax.axis('off')
plt.tight_layout()
plt.show()


Dice Loss (Loss function):
Hàm dice_loss được sử dụng làm hàm loss nhằm tối ưu mô hình để tăng cường sự trùng khớp giữa dự đoán và nhãn thực. Khi tối ưu hàm loss này, mục tiêu là làm giảm giá trị loss – nghĩa là cải thiện sự tương đồng giữa kết quả dự đoán và dữ liệu thật.

Dice Coefficient (Metric):
Đây là một chỉ số đánh giá (metric) thể hiện tỷ lệ giao nhau giữa dự đoán và nhãn thật. Chỉ số này thường dao động từ 0 đến 1, với giá trị càng cao thể hiện dự đoán càng tốt. Việc hiển thị Dice Coefficient trong quá trình huấn luyện giúp bạn theo dõi thêm một khía cạnh về hiệu quả mô hình ngoài accuracy.